# Preparação do dados
_Feature Engineering_ e _Construção de Target_

---

## Sumário

1. **Importação de bibliotecas**
2. **Carregamento das bases**
3. **Análise dos dataframes**
4. **Feature Engineering**
5. **Construção de Target**

<br>

---

<br>

## 1. Importação de bibliotecas

In [58]:
# Importação de pacotes e definição de parâmetros globais

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import missingno as msno
import warnings
import gc

In [59]:
# Configurações para exibição de dados no Jupyter Notebook

# Configurar opção para exibir todas as linhas do Dataframe
pd.set_option('display.max_rows', None)

# Configurar para exibir o conteúdo completo das colunas
pd.set_option('display.max_colwidth', None)

# Configurar a supressão de mensagens de aviso durante a execução
warnings.filterwarnings('ignore')

# Configurar estilo dos gráficos do seaborn
sns.set_style('whitegrid')

## 2. Carregamento das bases

In [ ]:
# Efetuando a limpeza da memória antes do carregamento dos dados
print(f'\nQuantidade de objetos removidos da memória: {gc.collect()}')

In [ ]:
# Criando um dataframe a partir do arquivo train.csv
df_train = pd.read_csv('dados/train.csv', sep=',')
print('\nDATAFRAME: df_train')
df_train.head()

In [ ]:
# Criando um dataframe a partir do arquivo test.csv
df_test = pd.read_csv('dados/test.csv', sep=',')
print('\nDATAFRAME: df_test')
df_test.head()

In [ ]:
# Criando um dataframe a partir do arquivo store.csv
df_store = pd.read_csv('dados/store.csv', sep=',')
print('\nDATAFRAME: df_store')
df_store.head()

## 3. Análise dos dataframes

In [ ]:
# Exibindo a quantidade de linhas e colunas dos dataframes

# Criação de um dicionário com os dataframes e seus respectivos nomes
dfs = {
    'df_train': df_train,
    'df_test': df_test,
    'df_store': df_store
}

# Iteração sobre o dicionário para exibir o nome e as dimensões dos dataframes
print(f'\nVOLUMETRIA')
for nome, df in dfs.items():
    print(f'\n{nome}')
    print(f'-'*45)
    print(f'Quantidade de linhas (registros):  {df.shape[0]}')
    print(f'Quantidade de colunas (variáveis): {df.shape[1]}')   

In [65]:
# Unindo df_train com df_store
df_train_full = pd.merge(df_train, df_store, on='Store', how='left')

# Unindo df_train com df_store
df_test_full = pd.merge(df_test, df_store, on='Store', how='left')

In [ ]:
df_train_full.head()

In [ ]:
df_test_full.head()

In [68]:
# Função para geração de um dataframe de metadados

def gerar_metadados(dataframe):
    '''
    Gera um dataframe contendo metadados das colunas do dataframe fornecido.

    :param dataframe: Dataframe
        DataFrame para o qual os metadados serão gerados.
    :return: DataFrame
        DataFrame contendo os metadados.
    '''
    metadados = pd.DataFrame({
        'Variável': dataframe.columns,
        'Tipo': dataframe.dtypes,
        'Qtde de nulos': dataframe.isnull().sum(),
        '% de nulos': round((dataframe.isnull().sum()/len(dataframe))*100, 2),
        'Cardinalidade': dataframe.nunique(),
    })
    metadados = metadados.sort_values(by='Qtde de nulos', ascending=False)
    metadados = metadados.reset_index(drop=True)
    return metadados

In [ ]:
# Exibindo os metadados do dataframe unificado de treino
gerar_metadados(df_train_full)

In [ ]:
# Colunas que estão em df_train_full mas não em df_test_full
diff_columns = df_train_full.columns.difference(df_test_full.columns)
diff_columns

In [71]:
# Transforma para o tipo de dados datetime
df_train_full['Date'] = pd.to_datetime(df_train_full['Date'])
df_test_full['Date'] = pd.to_datetime(df_test_full['Date'])

## 4. Feature Engineering

In [72]:
# Dicionário com os dataframes de treino e teste
dfs = {
    'df_train_full': df_train_full,
    'df_test_full': df_test_full
}

In [73]:
# Variáveis temporais

for name, df in dfs.items():
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['Quarter'] = df['Date'].dt.quarter
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['IsWeekend'] = df['Date'].dt.dayofweek >= 5
    df['DayOfYear'] = df['Date'].dt.dayofyear

In [74]:
# Variáveis de concorrência

def calculate_months_since_competition_opened(sub_df):
    '''
    Calcula o número de meses desde que a concorrência abriu para cada loja.
    
    :param sub_df: dataframe
        Um DataFrame contendo as colunas expecíficas
    :return: Series
        Uma série com o número de meses desde que a concorrência abriu, com
        valore maiores ou iguais a zero(0).
    ''' 
    months = (sub_df['Year'] - sub_df['CompetitionOpenSinceYear']) * 12 + (
        sub_df['Month'] - sub_df['CompetitionOpenSinceMonth'])
    return months.clip(lower=0)


def calculate_years_since_competition_opened(sub_df):
    '''
    Calcula o número de anos desde que a concorrência abriu para cada loja.
    
    :param sub_df: dataframe
        Um DataFrame contendo as colunas expecíficas
    :return: Series
        Uma série com o número de anos desde que a concorrência abriu, com
        valore maiores ou iguais a zero(0).
    ''' 
    years = (sub_df['Year'] - sub_df['CompetitionOpenSinceYear']) + (
        sub_df['Month'] - sub_df['CompetitionOpenSinceMonth']) / 12
    return years.clip(lower=0)

In [75]:
for name, df in dfs.items():

	df['MonthsSinceTheCompetitionOpened'] = (
		df.groupby('Store').apply(calculate_months_since_competition_opened
                            ).reset_index(drop=True))

	df['YearsSinceTheCompetitionOpened'] = (
		df.groupby('Store').apply(calculate_years_since_competition_opened
                            ).reset_index(drop=True))

In [76]:
# Variáveis de somas móveis

def add_rolling_sums(df, windows, vars):
    '''
    Adicionar variáveis de somas móveis.
    
    param df: Dataframe
        DataFrame onde as somas móveis serão adicionadas.
    param windows: list
        Lista de tamanhos de janelas móveis para calcular a soma.
    param vars:
        Lista contendo duas strings: a primeira é a variável para agrupar e a segunda
        é a variável sobre a qual calcular a soma móvel.
        
    return: Dataframe
        DataFrame original com as novas colunas de soma móvel adicionadas.
    '''
    for window in windows:
        name = f'{vars[1]}RollingSum_{window}'
        df[name] = df.groupby(vars[0])[vars[1]].transform(lambda x: x.rolling(window=window).sum())
    return df

In [77]:
windows = [3, 5, 7, 14, 21, 28, 30, 60, 90, 120, 150, 180, 360]
vars = ['Store', 'Promo']

df_train_full = add_rolling_sums(df_train_full, windows, vars)
df_test_full = add_rolling_sums(df_train_full, windows, vars)

## 5. Construção de target

In [ ]:
# Soma das vendas nos próximos 42 dias (6 semanas), excluindo o dia atual
df_train_full['Target'] = df_train_full.groupby('Store')['Sales'].transform(
    lambda x: x.rolling(window=42, min_periods=1).sum().shift(-42))

# Definir Target como zero para os dias com 'Sales' igual a zero
df_train_full['Target'] = np.where(df_train_full['Sales'] == 0, 0, df_train_full['Target'])

# Remover as linhas onde o Target não pode ser calculado (últimos 42 dias)
df_train_full.dropna(subset=['Target'], inplace=True)


In [ ]:
df_train_full.head()